
### Interactive Workflow of Principal Component Analysis, Eigen Values and Eigen Vectors
      

#### Principal Component Analysis

Principal Component Analysis one of a variety of methods for dimensional reduction:

Dimensional reduction transforms the data to a lower dimension

* Given features, $𝑋_1,\dots,𝑋_𝑚$  we would require ${m \choose 2}=\frac{𝑚 \cdot (𝑚−1)}{2}$ scatter plots to visualize just the two-dimensional scatter plots.

* Once we have 4 or more variables understanding our data gets very hard.
* Recall the curse of dimensionality, impact inference, modeling and visualization. 

One solution, is to find a good lower dimensional, $𝑝$,  representation of the original dimensions $𝑚$

Benefits of Working in a Reduced Dimensional Representation:

1. Data storage / Computational Time
2. Easier visualization
3. Also takes care of multicollinearity 

#### Orthogonal Transformation 

Convert a set of observations into a set of linearly uncorrelated variables known as principal components

* The number of principal components ($k$) available are min⁡($𝑛−1,𝑚$) 

* Limited by the variables/features, $𝑚$, and the number of data

Components are ordered

* First component describes the larges possible variance / accounts for as much variability as possible
* Next component describes the largest possible remaining variance 
* Up to the maximum number of principal components

Eigen Values / Eigen Vectors

* The Eigen values are the variance explained for each component
* The Eigen vectors of the data covariance matrix are the principal components' loadings

#### Install Packages

For this interactive workflow to work, we need to install several packages relating to display features, widgets and data analysis interpretation.

In [1]:
import pandas as pd                                       # DataFrames and plotting
import numpy as np
import matplotlib.pyplot as plt                           # plotting
from matplotlib.colors import ListedColormap              # custom color maps
import matplotlib.ticker as mtick
from matplotlib.patches import Rectangle
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numpy.linalg import eig                              # Eigen values and Eigen vectors
from sklearn.decomposition import PCA                     # PCA program from scikit learn (package for machine learning)
from sklearn.preprocessing import StandardScaler          # normalize synthetic data
from ipywidgets import interactive                        # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import warnings
warnings.filterwarnings('ignore')                         # ignore warnings
plt.rc('axes', axisbelow=True)                            # grids behind plot elements

#### Make the Dashbaord

The numerical methods for this dashboard are:

1. make a covariance matrix
2. sample jointly from the multiGaussian distribution based on this covariance matrix
3. standardize the samples to correct the mean and variance to 0.0 and 1.0 repsectively
4. calculate the actual covariance matrix / this ensures that the covariance matrix is positive semidefiniate (because it is based on actual data)
5. calculate the Eigen values and vectors, and sort by descending Eigen values
6. plot the original feature variances, Eigen vectors and values.

In [2]:
l = widgets.Text(value='                                         PCA Eigen Vector / Component Loadings Demo, Prof. Michael Pyrcz, The University of Texas at Austin',
        layout=Layout(width='900px', height='30px'))
# P_happening_label = widgets.Text(value='Probability of Happening',layout=Layout(width='50px',height='30px',line-size='0 px'))
cstr = widgets.FloatSlider(min=0.0, max = 1.0, value=0.0, step = 0.1, description = r'$\rho_{strength}$',orientation='horizontal', 
        style = {'description_width':'initial','button_color':'green'},layout=Layout(width='600px',height='40px'),continuous_update=False,readout_format='.3f')

ui_summary = widgets.HBox([cstr],)
ui_summary1 = widgets.VBox([l,ui_summary],)

def run_plot_summary(cstr):
    
    m = 4;
    
    mean = np.zeros((m))                         # make inputs for multivariate dataset
    #cov = np.zeros((m,m))
    cov = np.full((m,m),0.0)
    for i in range(0,m):
        cov[i,i] = 1.0
    cov[0,1] = cov[1,0] = 0.99*cstr; cov[1,2] = cov[2,1] = -0.9*cstr; cov[0,2] = cov[2,0] = -0.7*cstr;
    
    data = np.random.multivariate_normal(mean = mean, cov = cov, size = 1000) # draw samples from MV Gaussian
    data = StandardScaler(copy=True, with_mean=True, with_std=True).fit(data).transform(data)
    
    cov_actual = np.cov(data,rowvar = False)
    
    eigen_values,eigen_vectors = eig(cov_actual) # Eigen values and vectors 
    sorted_indices = np.argsort(-eigen_values)
    sorted_eigen_vectors = eigen_vectors[:, sorted_indices]
    sorted_eigen_values = np.sort(-eigen_values)*-1
    
    fig = plt.figure(figsize=(6, 6))
    gs = fig.add_gridspec(2,2 ,width_ratios=(1.0, 1.0))
    
    plt_center = fig.add_subplot(gs[1, 1])
    plt_x = fig.add_subplot(gs[1, 0],sharey=plt_center) 
    plt_y = fig.add_subplot(gs[0, 1],sharex=plt_center) 
    plt_extra = fig.add_subplot(gs[0, 0]) 
    
    for i in range(0,m):
        for j in range(0,m):
            color = (sorted_eigen_vectors[j,i] + 1.0)/(2.0)
            plt_center.add_patch(Rectangle((i-0.5,j-0.5), 1, 1,color = plt.cm.RdGy_r(color),fill=True))
            
            if abs(sorted_eigen_vectors[j,i]) > 0.5:
                plt_center.annotate(np.round(sorted_eigen_vectors[j,i],1),(i-0.1,j-0.05),color='white')
            else:
                plt_center.annotate(np.round(sorted_eigen_vectors[j,i],1),(i-0.1,j-0.05))
    
    plt_center.set_xlim([-0.5,3.5]); plt_center.set_ylim([-0.5,3.5])
    plt_center.set_xticks([0,1, 2, 3],[1,2,3,4]); plt_center.set_yticks([0,1, 2, 3],[1,2,3,4])
    for x in np.arange(0.5,3.5,1.0):
        plt_center.plot([x,x],[-0.5,3.5],c='black',lw=3)
        plt_center.plot([-0.5,3.5],[x,x],c='black',lw=1,ls='--')
    plt_center.set_title('Eigen Vectors / Principal Component Loadings')  
    plt_center.set_xlabel('Eigen Vector'); plt_center.set_ylabel('Feature')
    
    plt_x.barh(y=np.array([0,1,2,3],dtype='float'),width=np.var(data,axis=0),color='darkorange',edgecolor='black')
    plt_x.set_xlim([3.0,0]); plt_x.set_yticks([0,1, 2, 3],[1,2,3,4])
    plt_x.plot([1,1],[-0.5,3.5],c='black',ls='--'); plt_x.annotate('Equal Variance',(1.13,2.6),rotation=90.0,size=9)
    plt_x.set_ylabel('Feature'); plt_x.set_xlabel('Variance')
    plt_x.set_title('Original Feature Variance') 
    plt_x.grid(axis='x',which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
    plt_x.grid(axis='x',which='major', color='#DDDDDD', linewidth=0.8); plt_x.minorticks_on()
    for x in np.arange(0.5,3.5,1.0):
        plt_x.plot([-0.5,3.5],[x,x],c='black',lw=1,ls='--')
    
    plt_y.bar(x=np.array([0,1,2,3],dtype='float'),height=sorted_eigen_values,color='darkorange',edgecolor='black')
    plt_y.set_ylim([0,3.0]); plt_y.set_xticks([0,1, 2, 3],[1,2,3,4]); 
    plt_y.plot([-0.5,3.5],[1,1],c='black',ls='--'); plt_y.annotate('Equal Variance',(2.55,1.05),size=9)
    plt_y.set_xlabel('Eigen Value'); plt_y.set_ylabel('Variance')
    plt_y.set_title('Sorted, Projected Feature Variance') 
    plt_y.grid(axis='y',which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
    plt_y.grid(axis='y',which='major', color='#DDDDDD', linewidth=0.8); plt_y.minorticks_on()    
    for x in np.arange(0.5,3.5,1.0):
        plt_y.plot([x,x],[-0.5,3.5],c='black',lw=3)

    for i in range(0,m):
        for j in range(0,m):
            color = (cov_actual[j,i] + 1.0)/(2.0)
            plt_extra.add_patch(Rectangle((i-0.5,j-0.5), 1, 1,color = plt.cm.BrBG(color),fill=True))
    
    plt_extra.set_xlim([-0.5,3.5]); plt_extra.set_ylim([3.5,-0.5])
    plt_extra.set_xticks([0,1, 2, 3],[1,2,3,4]); plt_extra.set_yticks([0,1, 2, 3],[1,2,3,4])
    for x in np.arange(0.5,3.5,1.0):
        plt_extra.plot([x,x],[-0.5,3.5],c='black',lw=2)
        plt_extra.plot([-0.5,3.5],[x,x],c='black',lw=2)
    plt_extra.set_title('Covariance Matrix')  
     
    cplt_extra = make_axes_locatable(plt_extra).append_axes('left', size='5%', pad=0.3)
    fig.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=-1.0, vmax=1.0), cmap=plt.cm.BrBG),
                 cax=cplt_extra, orientation='vertical')
    cplt_extra.yaxis.set_ticks_position('left')
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1.51, top=1.50, wspace=0.2, hspace=0.2); plt.show()
    
interactive_plot_summary = widgets.interactive_output(run_plot_summary, {'cstr':cstr,})
interactive_plot_summary.clear_output(wait = True)  

### Interactive Principal Components Analysis, Component Loadings & Variance Explained Demostration

* add data correlation / redundancy and observe the impact on the component loadings (Eigen vectors) and variance explained (Eigen values).

The Inputs: **$\rho_{strenth}$**: the strength of the correlation between features, scaler applied to $X_1$, $X_2$, & $X_3$ correlation.

In [3]:
display(ui_summary1, interactive_plot_summary)                           # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'metadata': {}, 'data': {'text/plain': '<Figure size 600x600 w…

#### Comments

This was an interactive demonstrations of the Eigen values (variance explained) and Eigen vectors (component loadings) for Principal Components Analysis (PCA) with varianble between feature correlation. 